In [1]:
import numpy as np
import pandas as pd
from bs4 import BeautifulSoup
import requests
import json

In [2]:
url='https://en.wikipedia.org/wiki/List_of_postal_codes_of_Canada:_M'
request=requests.get(url).text

In [3]:
soup=BeautifulSoup(request,'html5lib')

In [4]:
table_contents=[]
table=soup.find('table')
for row in table.findAll('td'):
    cell = {}
    if row.span.text=='Not assigned':
        pass
    else:
        cell['PostalCode'] = row.p.text[:3]
        cell['Borough'] = (row.span.text).split('(')[0]
        cell['Neighborhood'] = (((((row.span.text).split('(')[1]).strip(')')).replace(' /',',')).replace(')',' ')).strip(' ')
        table_contents.append(cell)

In [5]:
df=pd.DataFrame(table_contents)
df['Borough']=df['Borough'].replace({'Downtown TorontoStn A PO Boxes25 The Esplanade':'Downtown Toronto Stn A',
                                             'East TorontoBusiness reply mail Processing Centre969 Eastern':'East Toronto Business',
                                             'EtobicokeNorthwest':'Etobicoke Northwest','East YorkEast Toronto':'East York/East Toronto',
                                             'MississaugaCanada Post Gateway Processing Centre':'Mississauga'})

In [6]:
df

,PostalCode,Borough,Neighborhood
0,M3A,North York,Parkwoods
1,M4A,North York,Victoria Village
2,M5A,Downtown Toronto,"Regent Park, Harbourfront"
3,M6A,North York,"Lawrence Manor, Lawrence Heights"
4,M7A,Queen's Park,Ontario Provincial Government
...,...,...,...
98,M8X,Etobicoke,"The Kingsway, Montgomery Road, Old Mill North"
99,M4Y,Downtown Toronto,Church and Wellesley
100,M7Y,East Toronto Business,Enclave of M4L
101,M8Y,Etobicoke,"Old Mill South, King's Mill Park, Sunnylea, Hu..."


In [12]:
import geocoder # import geocoder

# initialize your variable to None
lat_lng_coords = None

# loop until you get the coordinates
while(lat_lng_coords is None):
  g = geocoder.google('{}, Toronto, Ontario'.format(df.PostalCode))
  lat_lng_coords = g.latlng

latitude = lat_lng_coords[0]
longitude = lat_lng_coords[1]

KeyboardInterrupt: ignored

Geopy somehow took too long to respond.

So I used the csv file

In [8]:
from google.colab import files
files.upload()

Saving Geospatial_Coordinates.csv to Geospatial_Coordinates.csv


{'Geospatial_Coordinates.csv': b'Postal Code,Latitude,Longitude\nM1B,43.8066863,-79.1943534\nM1C,43.7845351,-79.1604971\nM1E,43.7635726,-79.1887115\nM1G,43.7709921,-79.2169174\nM1H,43.773136,-79.2394761\nM1J,43.7447342,-79.2394761\nM1K,43.7279292,-79.2620294\nM1L,43.7111117,-79.2845772\nM1M,43.716316,-79.2394761\nM1N,43.692657,-79.2648481\nM1P,43.7574096,-79.273304\nM1R,43.7500715,-79.2958491\nM1S,43.7942003,-79.2620294\nM1T,43.7816375,-79.3043021\nM1V,43.8152522,-79.2845772\nM1W,43.7995252,-79.3183887\nM1X,43.8361247,-79.2056361\nM2H,43.8037622,-79.3634517\nM2J,43.7785175,-79.3465557\nM2K,43.7869473,-79.385975\nM2L,43.7574902,-79.3747141\nM2M,43.789053,-79.4084928\nM2N,43.7701199,-79.4084928\nM2P,43.7527583,-79.4000493\nM2R,43.7827364,-79.4422593\nM3A,43.7532586,-79.3296565\nM3B,43.7459058,-79.352188\nM3C,43.7258997,-79.340923\nM3H,43.7543283,-79.4422593\nM3J,43.7679803,-79.4872619\nM3K,43.7374732,-79.4647633\nM3L,43.7390146,-79.5069436\nM3M,43.7284964,-79.4956974\nM3N,43.7616313,-79.

In [13]:
df1=pd.read_csv('Geospatial_Coordinates.csv')

In [14]:
df2=df.merge(df1, left_on='PostalCode', right_on='Postal Code')

In [15]:
df2.drop('Postal Code', axis=1, inplace=True)

In [16]:
df2

,PostalCode,Borough,Neighborhood,Latitude,Longitude
0,M3A,North York,Parkwoods,43.753259,-79.329656
1,M4A,North York,Victoria Village,43.725882,-79.315572
2,M5A,Downtown Toronto,"Regent Park, Harbourfront",43.654260,-79.360636
3,M6A,North York,"Lawrence Manor, Lawrence Heights",43.718518,-79.464763
4,M7A,Queen's Park,Ontario Provincial Government,43.662301,-79.389494
...,...,...,...,...,...
98,M8X,Etobicoke,"The Kingsway, Montgomery Road, Old Mill North",43.653654,-79.506944
99,M4Y,Downtown Toronto,Church and Wellesley,43.665860,-79.383160
100,M7Y,East Toronto Business,Enclave of M4L,43.662744,-79.321558
101,M8Y,Etobicoke,"Old Mill South, King's Mill Park, Sunnylea, Hu...",43.636258,-79.498509


Clustering areas based on the number of neighborhood that have the same postal code

In [17]:
from sklearn.cluster import KMeans

Data Manipulation

In [18]:
nneighbor=[]
for a in df.Neighborhood:
  nneighborhood=a.count(',')+1
  nneighbor.append(nneighborhood)

In [22]:
df2['nNeighbor']=nneighbor

In [23]:
df2

,PostalCode,Borough,Neighborhood,Latitude,Longitude,nNeighbor
0,M3A,North York,Parkwoods,43.753259,-79.329656,1
1,M4A,North York,Victoria Village,43.725882,-79.315572,1
2,M5A,Downtown Toronto,"Regent Park, Harbourfront",43.654260,-79.360636,2
3,M6A,North York,"Lawrence Manor, Lawrence Heights",43.718518,-79.464763,2
4,M7A,Queen's Park,Ontario Provincial Government,43.662301,-79.389494,1
...,...,...,...,...,...,...
98,M8X,Etobicoke,"The Kingsway, Montgomery Road, Old Mill North",43.653654,-79.506944,3
99,M4Y,Downtown Toronto,Church and Wellesley,43.665860,-79.383160,1
100,M7Y,East Toronto Business,Enclave of M4L,43.662744,-79.321558,1
101,M8Y,Etobicoke,"Old Mill South, King's Mill Park, Sunnylea, Hu...",43.636258,-79.498509,8


Modelling

In [24]:
model=KMeans(n_clusters=5,n_init=12)

In [25]:
features=df2[['Latitude','Longitude','nNeighbor']]

In [26]:
features

,Latitude,Longitude,nNeighbor
0,43.753259,-79.329656,1
1,43.725882,-79.315572,1
2,43.654260,-79.360636,2
3,43.718518,-79.464763,2
4,43.662301,-79.389494,1
...,...,...,...
98,43.653654,-79.506944,3
99,43.665860,-79.383160,1
100,43.662744,-79.321558,1
101,43.636258,-79.498509,8


In [27]:
model.fit(features)

KMeans(algorithm='auto', copy_x=True, init='k-means++', max_iter=300,
       n_clusters=5, n_init=12, n_jobs=None, precompute_distances='auto',
       random_state=None, tol=0.0001, verbose=0)

In [28]:
df2['Cluster']=model.labels_

In [29]:
df2

,PostalCode,Borough,Neighborhood,Latitude,Longitude,nNeighbor,Cluster
0,M3A,North York,Parkwoods,43.753259,-79.329656,1,2
1,M4A,North York,Victoria Village,43.725882,-79.315572,1,2
2,M5A,Downtown Toronto,"Regent Park, Harbourfront",43.654260,-79.360636,2,0
3,M6A,North York,"Lawrence Manor, Lawrence Heights",43.718518,-79.464763,2,0
4,M7A,Queen's Park,Ontario Provincial Government,43.662301,-79.389494,1,2
...,...,...,...,...,...,...,...
98,M8X,Etobicoke,"The Kingsway, Montgomery Road, Old Mill North",43.653654,-79.506944,3,4
99,M4Y,Downtown Toronto,Church and Wellesley,43.665860,-79.383160,1,2
100,M7Y,East Toronto Business,Enclave of M4L,43.662744,-79.321558,1,2
101,M8Y,Etobicoke,"Old Mill South, King's Mill Park, Sunnylea, Hu...",43.636258,-79.498509,8,1


Plotting in Map

In [30]:
import matplotlib.cm as cm
import matplotlib.colors as colors

In [31]:
nCluster=np.max(df2.Cluster)+1

In [32]:
import folium
toronto_lat=43.651070
toronto_lon=-79.347015
map_cluster=folium.Map([toronto_lat,toronto_lon],zoom_start=15)

In [33]:
ka = np.arange(nCluster)
ys = [i + ka + (i*ka)**2 for i in range(nCluster)]
colors_array = cm.rainbow(np.linspace(0, 1, len(ys)))
rainbow = [colors.rgb2hex(i) for i in colors_array]

In [34]:
for pc,name, lat,lon, cluster,nNeigh in zip(df2.PostalCode, df2.Neighborhood,df2.Latitude,df2.Longitude,df2.Cluster,df2.nNeighbor):
  label=folium.Popup('{}\n\t {}\n Neighborhood(s) located here'.format(pc,nNeigh))
  folium.CircleMarker([lat,lon], radius=15, color= rainbow[cluster-1], popup=label,fill=True).add_to(map_cluster)

End Result

In [35]:
map_cluster

Cluster Blue: 1 neighborhood per postal code

Cluster Red: 2 neighborhoods per postal code

Cluster Orange; 3 Neighborhoods per postal code

Cluster Light Greeen: 4-5 Neighborhoods per postal code

Cluster Purple: More than 5 Neighborhoods per postal code


Postal codes that have more than 3 neighborhoods tend to be located at the west outskirts of Toronto.

Interestingly, The east part do not have any postal code that have more than 3 neighborhoods.

